In [1]:
import tensorflow as tf

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# !pip install tensorflow-datasets

In [6]:
import tensorflow_datasets as tfds

In [7]:
import tensorflow_hub as hub

In [8]:
# from here: https://www.tensorflow.org/tutorials/keras/text_classification_with_hub
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /home/arman/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [9]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by

In [10]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [12]:
embedding = "https://tfhub.dev/google/universal-sentence-encoder/4"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 512), dtype=float32, numpy=
array([[-0.04922847, -0.05123926,  0.04913209, ...,  0.034378  ,
         0.06433065,  0.05493138],
       [-0.03361724, -0.05627659,  0.02549881, ..., -0.03751905,
         0.08010864,  0.02234846],
       [ 0.00494889,  0.0089879 , -0.04066751, ...,  0.051773  ,
         0.07442491,  0.0545832 ]], dtype=float32)>

In [13]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 512)               256797824 
_________________________________________________________________
dense (Dense)                (None, 16)                8208      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 256,806,049
Trainable params: 256,806,049
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=3,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/3
30/30 [==============================] - 32s 1s/step - loss: 0.5627 - accuracy: 0.7593 - val_loss: 0.4097 - val_accuracy: 0.8379
Epoch 2/3
30/30 [==============================] - 28s 927ms/step - loss: 0.2754 - accuracy: 0.9028 - val_loss: 0.3013 - val_accuracy: 0.8759
Epoch 3/3
30/30 [==============================] - 28s 941ms/step - loss: 0.0876 - accuracy: 0.9790 - val_loss: 0.2816 - val_accuracy: 0.8926


In [16]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 0.293
accuracy: 0.886
